In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.insert(0, '..')
import fashion.preprocessing as prep
# display all columns
pd.set_option('display.max_columns', None)

# show all results of the notebook not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import networkx as nx
from node2vec import Node2Vec

In [2]:
shops = prep.load_shops('../data/20200120_filiali.csv', extra_info=False)
shops.head()

sales17 = prep.load_sales('../data/20200120_sales17.csv', shops)
sales17.head()

,StoreKey,Franchise,StoreType,ZipCode,City
0,2d459c389e1a0561a2953b95cfdc0caa,True,Store,95024,ACIREALE
1,c93364d5937332fea742adf1bb0fd67e,False,Store,37010,AFFI
2,8316b325ea6fd3bdeb794ffb649eb556,False,Store,80021,AFRAGOLA
3,c359def3032833a2ebbc12c8948a18f3,False,Store,51031,AGLIANA
4,293e777d87dea6fe28930c91bb2f99fa,True,Store,86081,AGNONE


,StoreKey,ReceiptKey,Date,Hour,EAN,Volume,NetIncome
0,2d459c389e1a0561a2953b95cfdc0caa,160217,20161210,1318,2088988001033,1,19.95
1,2d459c389e1a0561a2953b95cfdc0caa,160253,20161210,1634,2088988001033,1,19.95
2,2d459c389e1a0561a2953b95cfdc0caa,160319,20161210,1911,2088988001057,1,19.95
3,2d459c389e1a0561a2953b95cfdc0caa,160330,20161210,1931,2088988001064,1,19.95
4,2d459c389e1a0561a2953b95cfdc0caa,160499,20161211,1747,2088986001042,1,19.95


In [3]:
sales17['Product'] = sales17.EAN.astype(str).str[:-3]
sales17['Week'] = pd.DatetimeIndex(sales17.Date.astype(str)).week 

In [4]:
Sales_per_store_sums = sales17.groupby(['Product', 'StoreKey','Week'])['Volume'].sum() 
Sales_per_store_means = Sales_per_store_sums.groupby(['Product', 'StoreKey']).mean()
max_val = Sales_per_store_means.max()

In [5]:
Product2store_graph = Sales_per_store_means.reset_index()
Product2store_graph.Volume = Product2store_graph.Volume/max_val

In [10]:
print(max_val)
print(Product2store_graph.head())

157.39473684210526
      Product                          StoreKey    Volume
0  2025443001  000f9b2404cb1bd4cc945cc49f0cbd51  0.006353
1  2025443001  014432f470a071454a276a9b05a66783  0.012707
2  2025443001  015486d0de8b946f4c4eb52f6b6c804a  0.029649
3  2025443001  017b48f3041a706945046fd95d162b6b  0.019060
4  2025443001  022a63b07c24d290cf6bf9baabad3d16  0.042886


In [11]:
Product2store_graph.to_csv(r'Clean_Product2store_graph.csv', header=False, index=False)

In [2]:

# Create a graph

fh=open("Clean_Product2store_graph.csv", 'r')
graph = nx.read_weighted_edgelist(fh,delimiter=',',nodetype = str)





# here's your stuff to do:
https://www.analyticsvidhya.com/blog/2019/11/graph-feature-extraction-deepwalk/

In [72]:
# Create an adjacency matrix

A = nx.adjacency_matrix(graph)

In [75]:
A_array = A.toarray()
# you should clear this array from memory after you get the dict and the cumsum

In [76]:
A_cumsum = np.cumsum(A_array,axis=1)

In [15]:
matrix_lengths = A.shape[0]

In [63]:
row_total_probabilities = [A[x].sum() for x in range(0,matrix_lengths)]
items = list(range(matrix_lengths))

dict = {}
for item, probability in zip(items, row_total_probabilities):
    dict[item] = probability

In [127]:
def transition(x):
    y = np.random.uniform(high=dict[x])
    z = np.argwhere(A_cumsum[x]>y)[0][0]
    return z


In [168]:
#size is how many walks, the range is how many steps in each walk

random_walker = np.random.randint(0,matrix_lengths, size = 1000) 

#change it so you do a walk for every node 
# rather than randomly choosing nodes to walk with 

row = random_walker

for null in range(0,10):
    next_node =  np.asarray([transition(oof) for oof in row])
    random_walker = np.vstack((random_walker, next_node))
    row = random_walker[:][-1]
    
    
    
    
    

In [169]:
random_walker[:,0]

array([8227,  157, 4291,   61, 5082,   90, 1407,  166, 1905,  113,  941],
      dtype=int64)

In [ ]:
#Initialise your random walker

In [8]:
#so yeah there are only like 9000-ish products that were ever sold in stores - what's going on with that. 

<9219x9219 sparse matrix of type '<class 'numpy.float64'>'
	with 2987666 stored elements in Compressed Sparse Row format>

In [14]:

# Create a graph

fh=open("Clean_Product2store_graph.csv", 'r')
graph = nx.read_weighted_edgelist(fh,delimiter=',',nodetype = str)


# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(graph, dimensions=10, walk_length=30, num_walks=100, workers=1)  # Use temp_folder for big graphs

model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)


model.wv.save_word2vec_format(r'product2store_graph.emb')



Computing transition probabilities:   0%|          | 0/9219 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 1/9219 [00:06<17:40:52,  6.91s/it]

Computing transition probabilities:   0%|          | 2/9219 [00:12<16:47:49,  6.56s/it]

Computing transition probabilities:   0%|          | 3/9219 [00:18<16:22:59,  6.40s/it]

Computing transition probabilities:   0%|          | 4/9219 [00:24<15:32:59,  6.07s/it]

Computing transition probabilities:   0%|          | 5/9219 [00:30<15:53:01,  6.21s/it]

Computing transition probabilities:   0%|          | 6/9219 [00:39<17:43:38,  6.93s/it]

Computing transition probabilities:   0%|          | 7/9219 [00:46<17:55:36,  7.01s/it]

Computing transition probabilities:   0%|          | 8/9219 [00:52<17:06:06,  6.68s/it]

Computing transition probabilities:   0%|          | 9/9219 [00:59<17:24:27,  6.80s/it]

Computing transition probabilities:   0%|          | 10/9219 [01:04<16:12:58,  6.34s/it]

Computing transition probabil

Computing transition probabilities:   2%|▏         | 169/9219 [15:59<15:49:23,  6.29s/it]

Computing transition probabilities:   2%|▏         | 170/9219 [16:06<16:28:11,  6.55s/it]

Computing transition probabilities:   2%|▏         | 171/9219 [16:09<14:16:38,  5.68s/it]

Computing transition probabilities:   2%|▏         | 172/9219 [16:16<15:20:53,  6.11s/it]

Computing transition probabilities:   2%|▏         | 173/9219 [16:23<15:38:24,  6.22s/it]

Computing transition probabilities:   2%|▏         | 174/9219 [16:26<13:28:57,  5.37s/it]

Computing transition probabilities:   2%|▏         | 175/9219 [16:33<14:14:45,  5.67s/it]

Computing transition probabilities:   2%|▏         | 176/9219 [16:38<13:45:17,  5.48s/it]

Computing transition probabilities:   2%|▏         | 177/9219 [16:45<15:22:56,  6.12s/it]

Computing transition probabilities:   2%|▏         | 178/9219 [16:52<15:49:54,  6.30s/it]

Computing transition probabilities:   2%|▏         | 179/9219 [16:59<16:33:17,  6.59s/it]


Computing transition probabilities:   4%|▎         | 343/9219 [32:48<17:56:32,  7.28s/it]

Computing transition probabilities:   4%|▎         | 344/9219 [32:55<18:07:08,  7.35s/it]

Computing transition probabilities:   4%|▎         | 345/9219 [33:03<18:01:39,  7.31s/it]

Computing transition probabilities:   4%|▍         | 346/9219 [33:10<17:53:37,  7.26s/it]

Computing transition probabilities:   4%|▍         | 347/9219 [33:17<17:33:38,  7.13s/it]

Computing transition probabilities:   4%|▍         | 348/9219 [33:24<17:58:06,  7.29s/it]

Computing transition probabilities:   4%|▍         | 349/9219 [33:32<18:18:50,  7.43s/it]

Computing transition probabilities:   4%|▍         | 350/9219 [33:39<18:15:07,  7.41s/it]

Computing transition probabilities:   4%|▍         | 351/9219 [33:47<18:10:27,  7.38s/it]

Computing transition probabilities:   4%|▍         | 352/9219 [33:54<18:08:25,  7.37s/it]

Computing transition probabilities:   4%|▍         | 353/9219 [34:01<18:07:01,  7.36s/it]


Computing transition probabilities:   6%|▌         | 515/9219 [53:03<18:07:23,  7.50s/it]

Computing transition probabilities:   6%|▌         | 516/9219 [53:11<18:10:18,  7.52s/it]

Computing transition probabilities:   6%|▌         | 517/9219 [53:17<17:20:07,  7.17s/it]

Computing transition probabilities:   6%|▌         | 518/9219 [53:23<16:42:23,  6.91s/it]

Computing transition probabilities:   6%|▌         | 519/9219 [53:30<16:29:17,  6.82s/it]

Computing transition probabilities:   6%|▌         | 520/9219 [53:37<16:26:32,  6.80s/it]

Computing transition probabilities:   6%|▌         | 521/9219 [53:44<16:28:03,  6.82s/it]

Computing transition probabilities:   6%|▌         | 522/9219 [53:50<16:23:10,  6.78s/it]

Computing transition probabilities:   6%|▌         | 523/9219 [53:56<15:50:37,  6.56s/it]

Computing transition probabilities:   6%|▌         | 524/9219 [54:03<15:45:31,  6.52s/it]

Computing transition probabilities:   6%|▌         | 525/9219 [54:09<15:41:57,  6.50s/it]


Computing transition probabilities:   7%|▋         | 683/9219 [1:10:37<18:02:56,  7.61s/it]

Computing transition probabilities:   7%|▋         | 684/9219 [1:10:45<18:12:25,  7.68s/it]

Computing transition probabilities:   7%|▋         | 685/9219 [1:10:53<18:16:11,  7.71s/it]

Computing transition probabilities:   7%|▋         | 686/9219 [1:11:00<18:09:52,  7.66s/it]

Computing transition probabilities:   7%|▋         | 687/9219 [1:11:08<18:21:53,  7.75s/it]

Computing transition probabilities:   7%|▋         | 688/9219 [1:11:16<18:19:58,  7.74s/it]

Computing transition probabilities:   7%|▋         | 689/9219 [1:11:23<17:53:20,  7.55s/it]

Computing transition probabilities:   7%|▋         | 690/9219 [1:11:32<18:27:53,  7.79s/it]

Computing transition probabilities:   7%|▋         | 691/9219 [1:11:40<18:42:41,  7.90s/it]

Computing transition probabilities:   8%|▊         | 692/9219 [1:11:48<18:56:17,  8.00s/it]

Computing transition probabilities:   8%|▊         | 693/9219 [1:11:56

...

Computing transition probabilities:  48%|████▊     | 4427/9219 [10:56:08<17:20:05, 13.02s/it]

Computing transition probabilities:  48%|████▊     | 4428/9219 [10:56:23<18:19:49, 13.77s/it]

Computing transition probabilities:  48%|████▊     | 4429/9219 [10:56:39<19:11:54, 14.43s/it]

Computing transition probabilities:  48%|████▊     | 4430/9219 [10:56:54<19:30:32, 14.67s/it]

Computing transition probabilities:  48%|████▊     | 4431/9219 [10:57:11<20:28:52, 15.40s/it]

Computing transition probabilities:  48%|████▊     | 4432/9219 [10:57:27<20:26:01, 15.37s/it]

Computing transition probabilities:  48%|████▊     | 4433/9219 [10:57:40<19:42:45, 14.83s/it]

Computing transition probabilities:  48%|████▊     | 4434/9219 [10:57:53<18:56:38, 14.25s/it]

Computing transition probabilities:  48%|████▊     | 4435/9219 [10:58:08<19:06:05, 14.37s/it]

Computing transition probabilities:  48%|████▊     | 4436/9219 [10:58:22<18:50:57, 14.19s/it]

Computing transition probabilities:  48%|████▊    

Computing transition probabilities:  50%|████▉     | 4589/9219 [11:31:42<17:32:03, 13.63s/it]

Computing transition probabilities:  50%|████▉     | 4590/9219 [11:31:58<18:22:50, 14.29s/it]

Computing transition probabilities:  50%|████▉     | 4591/9219 [11:31:58<12:57:38, 10.08s/it]

Computing transition probabilities:  50%|████▉     | 4592/9219 [11:32:14<15:15:41, 11.87s/it]

Computing transition probabilities:  50%|████▉     | 4593/9219 [11:32:29<16:35:27, 12.91s/it]

Computing transition probabilities:  50%|████▉     | 4594/9219 [11:32:44<17:10:17, 13.37s/it]

Computing transition probabilities:  50%|████▉     | 4595/9219 [11:32:58<17:25:12, 13.56s/it]

Computing transition probabilities:  50%|████▉     | 4596/9219 [11:32:58<12:17:53,  9.58s/it]

Computing transition probabilities:  50%|████▉     | 4597/9219 [11:33:14<14:36:26, 11.38s/it]

Computing transition probabilities:  50%|████▉     | 4598/9219 [11:33:29<16:18:34, 12.71s/it]

Computing transition probabilities:  50%|████▉    

Computing transition probabilities:  52%|█████▏    | 4751/9219 [12:00:33<19:34:11, 15.77s/it]

Computing transition probabilities:  52%|█████▏    | 4752/9219 [12:00:50<19:50:07, 15.99s/it]

Computing transition probabilities:  52%|█████▏    | 4753/9219 [12:01:05<19:20:08, 15.59s/it]

Computing transition probabilities:  52%|█████▏    | 4754/9219 [12:01:18<18:23:38, 14.83s/it]

Computing transition probabilities:  52%|█████▏    | 4755/9219 [12:01:30<17:35:54, 14.19s/it]

Computing transition probabilities:  52%|█████▏    | 4756/9219 [12:01:45<17:38:55, 14.24s/it]

Computing transition probabilities:  52%|█████▏    | 4757/9219 [12:01:59<17:33:58, 14.17s/it]

Computing transition probabilities:  52%|█████▏    | 4758/9219 [12:02:14<17:51:19, 14.41s/it]

Computing transition probabilities:  52%|█████▏    | 4759/9219 [12:02:30<18:41:11, 15.08s/it]

Computing transition probabilities:  52%|█████▏    | 4760/9219 [12:02:46<19:00:42, 15.35s/it]

Computing transition probabilities:  52%|█████▏   

Computing transition probabilities:  53%|█████▎    | 4913/9219 [12:45:06<39:41:03, 33.18s/it]

Computing transition probabilities:  53%|█████▎    | 4914/9219 [12:45:44<41:30:08, 34.71s/it]

Computing transition probabilities:  53%|█████▎    | 4915/9219 [12:46:20<41:53:40, 35.04s/it]

Computing transition probabilities:  53%|█████▎    | 4916/9219 [12:46:56<42:06:20, 35.23s/it]

Computing transition probabilities:  53%|█████▎    | 4917/9219 [12:47:28<41:15:13, 34.52s/it]

Computing transition probabilities:  53%|█████▎    | 4918/9219 [12:48:02<41:01:14, 34.33s/it]

Computing transition probabilities:  53%|█████▎    | 4919/9219 [12:48:38<41:22:13, 34.64s/it]

Computing transition probabilities:  53%|█████▎    | 4920/9219 [12:49:12<41:06:36, 34.43s/it]

Computing transition probabilities:  53%|█████▎    | 4921/9219 [12:49:47<41:16:16, 34.57s/it]

Computing transition probabilities:  53%|█████▎    | 4922/9219 [12:50:20<41:01:48, 34.37s/it]

Computing transition probabilities:  53%|█████▎   

KeyboardInterrupt: 